In [ ]:
import pandas as pd


# pandas支持

* pandas 在 2.0 的时候，可以采用 pyarrow 作为后端。
    1.  之前的版本Pandas 的数据在内存中基本都是以 Numpy 数组的形式存在
    2.  Numpy 本身并不是为 DataFrame 而设计，对于一些数据类型的支持并不是很好。例如 NaN，NaT，pd.NA
* Arrow 是更契合的内存数据结构，不仅速度更快，也更省内存，对数据类型的支持也更好
* https://pandas.pydata.org/docs/user_guide/pyarrow.html

In [ ]:
import pyarrow

In [ ]:
arr1 = pyarrow.array([1, 2, 3])
print(arr1.__class__)

In [ ]:
arr2 = pyarrow.array([1, 2, 3], type=pyarrow.int32())
print(arr2.__class__)

In [ ]:
# pandas 在存储数据时默认使用 Numpy Array
pd.Series([1, 2, 3], dtype="int64")


In [ ]:
# 指定 Pyarrow，表示使用 Arrow 格式来存储数据
pd.Series([1, 2, 3], dtype="int64[pyarrow]")


## 设置默认的IO引擎

In [ ]:
# https://pandas.pydata.org/docs/user_guide/options.html
pd.options.io.parquet.engine = "pyarrow"

# 基础读写

In [ ]:

df = pd.DataFrame({
    "name": ["satori", "koishi", "marisa", "cirno"],
    "age": [17, 16, 18, 40],
    "gender": ["female"] * 4
})

df.to_parquet(
    "base.parquet.gz",
    # 需要 pip install pyarrow
    engine="pyarrow",
    # 压缩方式，可选择：'snappy'(默认), 'gzip', 'brotli', None
    compression="gzip",
    # 是否把 DataFrame 自带的索引写入，True
    # 但要注意的是，索引会以 range 对象的形式写入到元数据中,不会占用太多空间
    # 因此，并且速度还更快
    index=False
)

In [ ]:
df = pd.read_parquet("base.parquet.gz",
                     engine="pyarrow")
df

# 分区

* girl.parquet.gz 不再是文件，而是一个目录，然后目录里面会出现 4 个子目录。
* 因为我们是按照 name 分区的，而 name 有 4 个不同的值
* 只有那些专门用于分类、元素重复率非常高的字段，才适合做分区字段，最典型的就是日期。

In [ ]:
df = pd.DataFrame({
    "name": ["satori", "koishi", "marisa", "cirno"] * 2,
    "age": [17, 16, 18, 40] * 2,
    "gender": ["female"] * 8
})

# 
df.to_parquet(
    "partition.parquet.gz",
    engine="pyarrow",
    compression="gzip",
    # 按照 "name" 字段分区
    partition_cols=["name"]
)
df

In [ ]:
df = pd.read_parquet("partition.parquet.gz",
                     engine="pyarrow")
df

## 部分读取

读取分区后的部分记录

In [ ]:
# 读取name=satori的记录
pd.read_parquet("partition.parquet.gz/name=satori",
                     engine="pyarrow")

## 多分区

In [ ]:
import numpy as np
df = pd.DataFrame({
    "p1": ["a"] * 4 + ["b"] * 4 + ["c"] * 4,
    "p2": ["X", "X", "Y", "Y"] * 3,
    "p3": np.random.randint(1, 100, size=(12,))
})

df.to_parquet(
    "many_partition.parquet.gz",
    engine="pyarrow",
    compression="gzip",
    # 按照 "p1" 和 "p2" 字段分区
    partition_cols=["p1", "p2"]
)
df

### 读取多分区

In [ ]:
pd.read_parquet("many_partition.parquet.gz/p1=b",
                     engine="pyarrow")

In [ ]:
pd.read_parquet("many_partition.parquet.gz/p1=b/p2=X",
                     engine="pyarrow")